#Change semicolon delimiter to comma

NOTE: If you have any questions about the code, please contact rdr-contacte@csuc.cat

## PURPOSE OF THE SCRIPT

The main purpose of this script is to transform the semicolon delimiter of CSV files to comma.

In [ ]:
# @title Upload the CSV files
import os
import csv
import shutil
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from io import StringIO

# Function to process the CSV file
def process_csv(csv_content):
    output_csv = StringIO()

    csv_reader = csv.reader(StringIO(csv_content), delimiter=';')
    csv_writer = csv.writer(output_csv, delimiter=',', quoting=csv.QUOTE_MINIMAL)

    for row in csv_reader:
        csv_writer.writerow(row)

    return output_csv.getvalue()

# Function to handle file uploads
def on_file_upload(change):
    clear_output(wait=True)

    # Check if any files are uploaded
    if not change.new:
        print("Please upload a file.")
        return

    uploaded_files = change.new.values()

    # Process each uploaded file
    for uploaded_file in uploaded_files:
        file_content = uploaded_file['content'].decode('utf-8')
        original_filename = uploaded_file['metadata']['name']
        new_filename = os.path.splitext(original_filename)[0] + '_new.csv'

        # Process the CSV content
        processed_csv_content = process_csv(file_content)

        # Save the processed CSV content to a file
        with open(new_filename, 'w', encoding='utf-8') as f:
            f.write(processed_csv_content)

    # Create a directory to store processed files
    directory_name = 'processed_csv_files'
    os.makedirs(directory_name, exist_ok=True)

    # Move processed files to the directory
    for uploaded_file in uploaded_files:
        original_filename = uploaded_file['metadata']['name']
        new_filename = os.path.splitext(original_filename)[0] + '_new.csv'
        shutil.move(new_filename, os.path.join(directory_name, new_filename))

    # Create a zip file of the directory
    zip_file = shutil.make_archive(directory_name, 'zip', directory_name)

    # Trigger the download of the zip file
    files.download(zip_file)

# Displaying file upload message
display(HTML("<p style='font-size:14px;'><b>Please upload your semicolon-separated CSV files.</b></p>"))

# Creating upload button
upload_button = widgets.FileUpload(accept='.csv', multiple=True)
upload_button.observe(on_file_upload, names='value')

# Displaying the upload button
display(upload_button)
